# Device Classification Setup (Setup Instructions)

## Environment Setup

- Make sure you prepared the .env file in the root directory of the project. (Check the README for default values)

### Variables

#### Load Variables from .env File in current directory

In [ ]:
$varlist = @()
get-content .env | foreach {
    $name, $value = $_.split('=').Trim().Replace('"', '')
    if (!([string]::IsNullOrWhiteSpace($name) -or $name.Contains('#'))) {
      $varlist += @(New-Object PSObject -Property @{Name=$name;Value=$value})
      Set-Variable -name $name -value $value
    }
}
$PubCertFilePath = ".\Certificate\$($PubCertifcateFileName)"
$PrivCertFilePath = ".\Certificate\$($PrivCertifcateFileName)"
$varlist += @(New-Object PSObject -Property @{Name="PubCertFilePath";Value=$PubCertFilePath})
$varlist += @(New-Object PSObject -Property @{Name="PrivCertFilePath";Value=$PubCertFilePath})
$varlist | select Name, Value | ft -AutoSize
Remove-Variable name, value, varlist

### Uninstall Az Modules (optional)

In [ ]:
# Uninstall All Az Modules
Uninstall-Module -Name Az

# Uninstall Microsoft.Graph.Authentication Module
Uninstall-Module -Name Microsoft.Graph.Authentication

### Install Powershell Modules

In [ ]:
Install-Module -Name Az.Accounts -Repository PSGallery -Force
Install-Module -Name Az.Resources -Repository PSGallery -Force
Install-Module -Name Az.Automation -Repository PSGallery -Force
Install-Module -Name Microsoft.Graph.Authentication -Repository PSGallery -Force

In [ ]:
Get-Module -ListAvailable -Name Az* | Select-Object Name,Version
Get-Module -ListAvailable -Name Microsoft.Graph.Authentication | Select-Object Name,Version

## Intune Remediation Script

### Connect GraphAPI

In [ ]:
Connect-MgGraph -TenantId $TenantID -NoWelcome
Get-MgContext

### Create Intune Remediation Script Package

In [ ]:
$EncodedDetectionScript = [convert]::ToBase64String((Get-Content '.\Remediation Script\detect.ps1' -AsByteStream))
$RemediationScriptUri = "https://graph.microsoft.com/beta/deviceManagement/deviceHealthScripts"
$DetectionScriptPackage = @"
{
  "@odata.type": "#microsoft.graph.deviceHealthScript",
  "publisher": "$RemediationScriptPublisher",
  "version": "1.0",
  "displayName": "$RemediationScriptName",
  "description": "Uses only a detection script. It detects the device classification and outputs the classification as preRemediationDetectionScriptOutput property. This property is later being used in a Azure Automation Runbook to set the Entra ID device extensionAttribute1-15",
  "detectionScriptContent": "$EncodedDetectionScript",
  "runAsAccount": "system",
  "runAs32Bit": "False"
}
"@
$RemediationScript = Invoke-MgGraphRequest -Uri $RemediationScriptUri -Method POST -Body $DetectionScriptPackage
$RemediationScriptID = $RemediationScript.id
$RemediationScript | select displayName, publisher,runAsAccount, runAs32Bit, id

### Deploy the Remediation Script Package

In [ ]:
$IntunePortalLinkRS = "https://intune.microsoft.com/#view/Microsoft_Intune_Enrollment/UXAnalyticsScriptMenu/~/overview/id/$RemediationScriptID/scriptName/$RemediationScriptName/isFirstParty~/false"
$IntunePortalLinkRS = [URI]::EscapeUriString($IntunePortalLinkRS)
Write-Host "Goto: ($IntunePortalLinkRS)"

Goto the link above and deploy the Remediation Script Package to your target devices.

## Certificate




There are two ways to create a certificate for the device:
 - Create a self-signed certificate
 - Create a certificate signed by a CA

### New Self-Signed Certificate (Option 1)

#### Set Password

In [ ]:
#!set --name CertifcatePassword --value @password:"Please enter a Certificate password"

#### Create new self-signed certificate

In [ ]:

$NewSelfSignedCertificate = New-SelfSignedCertificate -Subject "CN=$CertifcateName" -CertStoreLocation "Cert:\CurrentUser\My" -KeyExportPolicy Exportable -KeySpec Signature -KeyLength 2048 -KeyAlgorithm RSA -HashAlgorithm SHA256
$NewSelfSignedCertificate

#### Export Certificate (without private key) to file

In [ ]:
$PubCertFile = Export-Certificate -Cert $NewSelfSignedCertificate -FilePath $PubCertFilePath
$PubCertFile

#### Export Certificate (with private key)

In [ ]:
$mypwd = ConvertTo-SecureString -String $CertifcatePassword -Force -AsPlainText 
$PrivCertFile = Export-PfxCertificate -Cert $NewSelfSignedCertificate -FilePath $PrivCertFilePath -Password $mypwd
$PrivCertFile

#### Remove Certificate

In [ ]:
$NewSelfSignedCertificate.Thumbprint

In [ ]:
Remove-Item -Path Cert:\CurrentUser\My\$($NewSelfSignedCertificate.Thumbprint) -DeleteKey

### Get CA Certificate (Option 2)

Place the CA certificates in Certificate folder. Rename according to values of PubCertifcateFileName and PrivCertifcateFileName frome .env file.

In [ ]:
Write-Host "PubCertFilePath : $(Test-Path -Path $PubCertFilePath)"
Write-Host "PrivCertFilePath: $(Test-Path -Path $PrivCertFilePath)"

## App Registration

### Connect to Azure Subscription

In [ ]:
Connect-AzAccount -Tenant $TenantID -Subscription $SubscriptionID

### Create App Registration

In [ ]:
$AppRegistration = New-AzADApplication -DisplayName $AppRegistrationName #-RequiredResourceAccess $RequiredResourceAccess
$AppRegistration | Select DisplayName, AppId, Id

### Add API Permissions

| **Scope** | **Type** | **Permission** | **Id** | **Resource App ID** |
|-----------|----------|----------------|------------------------|------------------------|
| User.Read | Delegated | Sign in and read user profile | e1fe6dd8-ba31-4d61-89e7-88639da4683d | 00000003-0000-0000-c000-000000000000 (Microsoft Graph) |
| Device.ReadWrite.All | Application | Read and write devices | 1138cb37-bd11-4084-a2b7-9f71582aeddb  | 00000003-0000-0000-c000-000000000000 (Microsoft Graph) |
| DeviceManagementConfiguration.Read.All | Application | Read Microsoft Intune device configuration and policies | dc377aa6-52d8-4e23-b271-2a7ae04cedf3 | 00000003-0000-0000-c000-000000000000 (Microsoft Graph) |
| DeviceManagementManagedDevices.Read.All | Application | Read Microsoft Intune devices | f51be20-0bb4-4fed-bf7b-db946066c75e | 00000003-0000-0000-c000-000000000000 (Microsoft Graph) |

#### Add API Permissions to Azure AD Application

In [ ]:
# User.Read
Add-AzADAppPermission `
    -ObjectId $AppRegistration.Id `
    -ApiId 00000003-0000-0000-c000-000000000000  `
    -PermissionId e1fe6dd8-ba31-4d61-89e7-88639da4683d  `
    -Type Scope

# Device.ReadWrite.All
Add-AzADAppPermission  `
    -ObjectId $AppRegistration.Id `
    -ApiId 00000003-0000-0000-c000-000000000000  `
    -PermissionId 1138cb37-bd11-4084-a2b7-9f71582aeddb  `
    -Type Role

# DeviceManagementConfiguration.Read.All
Add-AzADAppPermission  `
    -ObjectId $AppRegistration.Id `
    -ApiId 00000003-0000-0000-c000-000000000000  `
    -PermissionId dc377aa6-52d8-4e23-b271-2a7ae04cedf3  `
    -Type Role

# DeviceManagementConfiguration.ReadWrite.All
Add-AzADAppPermission  `
    -ObjectId $AppRegistration.Id `
    -ApiId 00000003-0000-0000-c000-000000000000  `
    -PermissionId 2f51be20-0bb4-4fed-bf7b-db946066c75e  `
    -Type Role

#### List Permissions

In [ ]:
Get-AzADAppPermission  -ObjectId $AppRegistration.Id

### Get Certificate and set to App Registration

In [ ]:
$cer = New-Object System.Security.Cryptography.X509Certificates.X509Certificate2 $PubCertFilePath
$CertValue = $cer.GetRawCertData()
$CertValue = [System.Convert]::ToBase64String($CertValue)
New-AzADAppCredential -CertValue $CertValue -StartDate $cer.NotBefore -EndDate $cer.NotAfter -ApplicationId $AppRegistration.AppId

### Grant Admin Consent on the Azure Portal

In [ ]:
$Uri = "https://aad.portal.azure.com/#view/Microsoft_AAD_RegisteredApps/ApplicationMenuBlade/~/CallAnAPI/appId/$($AppRegistration.AppId)/isMSAApp~/false"

Write-Host "Grant Admin Consent for $AppRegistrationName"
Write-Host "Review and Permission and Grant admin consent"
Write-Host "GoTo: $Uri"

Make sure you granted admin consent on the Entra ID app registration

## Automation Account

### Connect to Azure Subscription (optional, if not connected yet)

In [ ]:
Connect-AzAccount -Tenant $TenantID -Subscription $SubscriptionID

### Create Resource Group (optional)

In [ ]:
New-AzResourceGroup -Name $ResourceGroupName -Location $Location

### Create Automation Account

In [ ]:
$AzAutomationAccount = New-AzAutomationAccount -Name $AutomationAccountName -ResourceGroupName $ResourceGroupName -Location $Location -Plan $Plan
$AzAutomationAccount | Select AutomationAccountName , Location, State

### Add Certificate to Automation Account

In [ ]:
Write-Host $PrivCertFilePath
#!set --name CertPW --value @password:"Please enter the Certificate (pfx) $PrivCertFilePath password"
$Password = ConvertTo-SecureString -String $CertPW -AsPlainText -Force
$AzAutomationCertificate = New-AzAutomationCertificate `
    -AutomationAccountName $AutomationAccountName `
    -Name $CertifcateName `
    -Path $PrivCertFilePath `
    -Password $Password `
    -ResourceGroupName $ResourceGroupName
$AzAutomationCertificate

### Create Automation Account Connection

In [ ]:
$RunAsAccountConnectionFieldValues = @{
   "ApplicationId" = $AppRegistration.AppId; 
   "TenantId" = $TenantId; 
   "CertificateThumbprint" = $AzAutomationCertificate.Thumbprint;
   "SubscriptionId" = $SubscriptionID
}

$RunAsAccountConnectionFieldValues

New-AzAutomationConnection -Name $AppConnectionName  `
    -ConnectionTypeName AzureServicePrincipal `
    -ConnectionFieldValues $RunAsAccountConnectionFieldValues `
    -ResourceGroupName $ResourceGroupName `
    -AutomationAccountName $AutomationAccountName `
    -Description "Establishes a connection to the App Registration, with the certificate"

### Create Automation Account Variables

In [ ]:
New-AzAutomationVariable `
    -AutomationAccountName $AutomationAccountName `
    -Name "ClearValue" `
    -Value $ClearValue `
    -ResourceGroupName $ResourceGroupName `
    -Description "Define the value from the remediation script output that should clear the extensionAttribute, e.g. clear" `
    -Encrypted $False

New-AzAutomationVariable `
    -AutomationAccountName $AutomationAccountName `
    -Name "DeviceClassificationList" `
    -Value $DeviceClassificationList `
    -ResourceGroupName $ResourceGroupName `
    -Description "Comma separted list of Classifications. Make sure there are no spaces before and after the comma and have the clear value in the list as well e.g. CAD Device,Standard Device,Local Admin Device, clear" `
    -Encrypted $False

New-AzAutomationVariable `
    -AutomationAccountName $AutomationAccountName `
    -Name "ExtensionAttribute" `
    -Value $ExtensionAttribute `
    -ResourceGroupName $ResourceGroupName `
    -Description "Set the extensionAttribute1 to extensionAttribute15. Make sure it's not used for any other purpose. e.g. extensionAttribute1" `
    -Encrypted $False

New-AzAutomationVariable `
    -AutomationAccountName $AutomationAccountName `
    -Name "RemediationScriptID" `
    -Value $RemediationScriptID `
    -ResourceGroupName $ResourceGroupName `
    -Description "Id of the remediation script get it from https://graph.microsoft.com/beta/deviceManagement/deviceHealthScripts/" `
    -Encrypted $False

### Add Azure Automation Microsoft.Graph.Authentication Module

In [ ]:
New-AzAutomationModule  `
    -AutomationAccountName $AutomationAccountName  `
    -Name "Microsoft.Graph.Authentication"  `
    -ContentLink "https://devopsgallerystorage.blob.core.windows.net:443/packages/microsoft.graph.authentication.2.12.0.nupkg"  `
    -ResourceGroupName $ResourceGroupName  `
    -RuntimeVersion 7.2

In [ ]:
Get-AzAutomationModule  `
    -AutomationAccountName $AutomationAccountName  `
    -Name "Microsoft.Graph.Authentication"  `
    -ResourceGroupName $ResourceGroupName  `
    -RuntimeVersion 7.2 | Select Name, ProvisioningState

### Import Azure Automation Runbook

In [ ]:
Import-AzAutomationRunbook  `
    -AutomationAccountName $AutomationAccountName `
    -Name $RunBookName `
    -Path ".\AutomationAccount\Runbook.ps1" `
    -Published  `
    -ResourceGroupName $ResourceGroupName `
    -Type PowerShell72

### Schedule Azure Automation Runbook

In [ ]:
$TimeZone = ([System.TimeZoneInfo]::Local).Id
New-AzAutomationSchedule  `
    -AutomationAccountName $AutomationAccountName  `
    -Name "Daily"  `
    -StartTime $DailyScheduleTime  `
    -DayInterval 1  `
    -ResourceGroupName $ResourceGroupName  `
    -TimeZone $TimeZone

In [ ]:
Register-AzAutomationScheduledRunbook   `
    -RunbookName $RunBookName  `
    -ScheduleName "Daily"   `
    -AutomationAccountName $AutomationAccountName  `
    -ResourceGroupName $ResourceGroupName